In [72]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pathlib import Path
from tratamento import trata_acomph, a0_a1

In [73]:
def get_prod():
    loc = Path('../ex_csv/produtibilidades/prod.csv')
    prod = pd.read_csv(loc, index_col=0)
    for i,row in prod.head(154).iterrows():
        row['prod'] = row['prod'].replace(",",".")
        row['prod'] = float(row['prod'])
    return prod

In [74]:
def importa_arquivos():
    acomph = trata_acomph.trata()
    produtibilidade = get_prod()
    a0, a1 = a0_a1.separa_a0_a1()
    local = Path('../ex_csv')
    local_post = local / 'postos.csv'
    postos = pd.read_csv(local_post, index_col=0)
    return acomph, a0, a1, postos, produtibilidade

In [75]:
def get_datas():
    meses = []
    for i in range(1,31):
        meses = [((date.today()-timedelta(days=i+3)).month)] + meses
    return meses

In [76]:
def ena_tipo_0(ena, acomph, produtibilidades, postos):
    vazoes_tipo0 = acomph.join(postos.query('tipo==0'),
                               on = 'posto', how = 'inner')
    vazoes_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo0 = produtibilidades.join(postos.query('tipo==0'),
                                                  on = 'posto', how = 'inner')
    produtibilidade_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                               axis=1, inplace=True)
    ena_tipo0 = ena.join(postos.query('tipo==0'),
                         on = 'posto', how = 'inner')
    ena_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    for i in range(30):
        ena_dia =  vazoes_tipo0.iloc[:,i].multiply(produtibilidade_tipo0['prod'],
                                                  fill_value=0)
        ena.iloc[:,i] += ena_dia
    ena.fillna(0,inplace = True)
    return ena

In [91]:
def ena_tipo_1(ena, acomph, produtibilidades, postos, a0, a1):
    meses = get_datas()
    vazoes_tipo1 = acomph.join(postos.query('tipo==1'),
                               on = 'posto', how = 'inner')
    vazoes_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo1 = produtibilidades.join(postos.query('tipo==1'),
                                                 on='posto', how='inner')
    produtibilidade_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                              axis=1, inplace=True)
    vazoes_base = acomph.loc[a1.iloc[:,0],:]
    for i in range(30):
        ena.iloc[:,i] = a0.iloc[:,meses[i]] + a1.iloc[:,meses[i]] * vazoes_base.iloc[:,i]
    return ena

In [92]:
def cria_ena():
    acomph, a0, a1, postos, produtibilidades = importa_arquivos()
    ind = acomph.head(155).index
    col = (acomph.T).head(30).index
    ena = pd.DataFrame(0,index = ind, columns = col)
    ena = ena_tipo_0(ena,acomph,produtibilidades,postos)
    ena = ena_tipo_1(ena,acomph,produtibilidades,postos,a0,a1)
    return ena

In [94]:
cria_ena()

ValueError: cannot reindex from a duplicate axis